In [36]:
from _04_preprocessing import housing, housing_prepared, housing_labels
from _04_preprocessing import full_pipeline
from _04_preprocessing import start_test_set

In [15]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

LinearRegression()

# 선형회귀
제일 기본적인 **선형회귀** 모델

In [16]:
some_data = housing.iloc[:5]
some_prepared = full_pipeline.transform(some_data)
print(lin_reg.predict(some_prepared))

some_labels = housing_labels.iloc[:5]
print(list(some_labels))

[210644.60459286 317768.80697211 210956.43331178  59218.98886849
 189747.55849879]
[286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


임의의 데이터를 학습한 모델에 넣는 방법  
일단 성능이 형편없어 보인다;;

In [17]:
from sklearn.metrics import mean_squared_error
import numpy as np
lin_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, lin_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

68628.19819848922


학습에 쓰인 전체 데이터셋에 대한 성능 지표  
예측할 값인 중간 주택 가격의 범위가 120,000 ~ 265,000인데 이 정도면 거의 뭐 못 쓴다고 보면 된다.
---

In [18]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

DecisionTreeRegressor()

# 디시전트리
이번에는 **디시전트리** 모델을 이용

In [19]:
tree_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, tree_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)

0.0


??? 오차가 전혀 없다고 한다.  
말도 안된다고 생각한다면 맞는 말이다. 이것이 **오버피팅**이다.  
이런 경우를 확인하기 위하여 훈련세트를 또 쪼개서 **검증 세트**를 만드는 것이다.  
  
>훌륭한 방법인 **K-겹 교차 검증**을 사용해보자

In [20]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                        scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

**cv**매개변수는 몇 겹으로 쪼개서 할 것이냐를 나타냄
>**cross_val_score**클래스의 **scoring**매개변수는 비용함수(낮을수록 좋은)가 아니라  
>효용함수(높을수록 좋은)를 받기를 기대하기 때문에 에러에 -를 붙인 값을 사용한다.

In [21]:
def display_scores(scores):
    print("점수: ", scores)
    print("평균: ", scores.mean())
    print("표준편차: ", scores.std())

In [27]:
display_scores(tree_rmse_scores)

점수:  [68362.93967039 67385.18193727 73082.56972422 67675.36294787
 71688.32158736 74894.40892908 72008.96809207 71611.99777609
 76347.16740791 69307.70412735]
평균:  71236.46221996029
표준편차:  2886.883980878265


검증세트에 대해 성능을 보니 선형회귀 모델보다 형편없음을 확인할 수 있다.

In [28]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                            scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)

In [29]:
display_scores(lin_rmse_scores)

점수:  [66782.73843989 66960.118071   70347.95244419 74739.57052552
 68031.13388938 71193.84183426 64969.63056405 68281.61137997
 71552.91566558 67665.10082067]
평균:  69052.46136345083
표준편차:  2731.6740017983498


In [30]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

RandomForestRegressor()

# 랜덤포레스트
디시전트리의 **앙상블**버전인 **랜덤포레스트**모델 활용

In [32]:
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)

In [33]:
display_scores(forest_rmse_scores)

점수:  [49312.41406162 47960.72824476 49228.72252359 52378.31003832
 49740.04221191 53040.48766161 48554.21486233 48172.28323068
 53343.06696002 50344.70774369]
평균:  50207.49775385235
표준편차:  1910.2351889945664


In [34]:
forest_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, forest_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

18575.80571921274

위 셋 중에는 랜덤포레스트가 제일 낫지만 여전히 훈련셋에 대한 성능보다 검증셋에 대한 성능이 훨씬 떨어짐  
아직도 오버피팅이라는 얘기  
  
오버피팅을 방지하기 위한 방법으론
> **모델을 간단히 하기**  
> **학습시 제한(규제)를 가하기**  
> **더 많은 훈련 데이터를 모으기**  
  
여러 종류의 머신러닝 알고리즘으로 하이퍼파라미터 튜닝에 너무 많은 시간을 들이지 않으면서  
다양한 모델을 시도해봐야 함 (다양한 커널의 SVM, 신경망 등)  
가능한 2~5개 정도의 모델을 선정하는 것이 목적

In [35]:
import joblib
joblib.dump(forest_reg, "my_model.pkl")

'''
my_model_loaded = joblib.load(my_model.pkl)
'''

'\nmy_model_loaded = joblib.load(my_model.pkl)\n'

## 모델을 저장하고 불러오는 예시